In [1]:
%display latex
from delierium import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from IPython.core.debugger import set_trace
import itertools

In [2]:
def FindIntegrableConditions(S, context):
    l = [_l.Lder() for _l in S]
    for e1, e2 in itertools.product(S, repeat = 2):
        if bool(e1 == e2):
            continue
        e1_mult, e1_nonmult = multipliers(e1.Lder(),l, context._independent)
        e2_mult, e2_nonmult = multipliers(e2.Lder(),l, context._independent)
        print (e1, e1_mult, e1_nonmult)
        print (e2, e2_mult, e2_nonmult)
    return S

In [3]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [4]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
# ctx: items are in descending order ?
ctx = Context((w,z), (x,y), Mgrlex)
f1 =diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y
f1.show()

-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [5]:
f2=diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f2.show()

y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [6]:
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f3.show()

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [7]:
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2)
f4.show()

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

In [8]:
f5 =diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/(y**2)
f5.show()

-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [9]:
#f2a=f2-(y*f1/x);f2a.expand().show()

In [10]:
f1dp=Differential_Polynomial(f1, context=ctx)
f2dp=Differential_Polynomial(f2, context=ctx)
f3dp=Differential_Polynomial(f3, context=ctx)
f4dp=Differential_Polynomial(f4, context=ctx)
f5dp=Differential_Polynomial(f5, context=ctx)

In [11]:
class Janet_Basis:
    def __init__ (self, S, context):
        #self._S = JanetBasis(S,context)
        if not isinstance(S, Iterable):
            # bad criterion
            S = [S]
        self._init(S, context)
    def _init (self, S, context):
        S = Reorder([Differential_Polynomial(s, context) for s in S], context, ascending = True)
        while 1:
            S = Autoreduce (S, context)   
            print ("******************   Autoreduce ****************************************")
            for _ in S:
                print (_._p)
                _.show()            
            S = CompleteSystem(S, context)
            print ("******************   completesystem ****************************************")
            for _ in S:
                _.show()            
            S = FindIntegrableConditions(S, context)
            print ("******************   FindIntegr ****************************************")
            for _ in S:
                _.show()            
            S = c = ReduceIntegrabilityConditions(S, context)
            print ("******************   ReduceIntegr ****************************************")
            for _ in S:
                _.show()            
            if not c or all ([_ == 0 for _ in c]):
                self._S = S
                return
            S += c
            S = Reorder (S, context)
            self._S = S[:]        
            return 
            
    def show(self):
        for _ in self._S:
            _.show()

In [12]:
checkS1=Janet_Basis([f3,f1,f5, f2,f4], ctx)

******************   Autoreduce ****************************************


diff(z(x, y), y)

[<delierium.JanetBasis.DTerm object at 0x7efbda526ca0>, <delierium.JanetBasis.DTerm object at 0x7efbda5234c0>]


1/2*w(x, y)/y + diff(z(x, y), x)

[<delierium.JanetBasis.DTerm object at 0x7efbda85ad30>, <delierium.JanetBasis.DTerm object at 0x7efbdb24d8b0>]


...

******************   ReduceIntegr ****************************************


diff(z(x, y), y)

1/2*w(x, y)/y + diff(z(x, y), x)

-w(x, y)/y + diff(w(x, y), y)

diff(w(x, y), x)

In [13]:
checkS1.show()

diff(w(x, y), x)

diff(w(x, y), x)

-w(x, y)/y + diff(w(x, y), y)

-w(x, y)/y + diff(w(x, y), y)

1/2*w(x, y)/y + diff(z(x, y), x)

1/2*w(x, y)/y + diff(z(x, y), x)

diff(z(x, y), y)

diff(z(x, y), y)

In [14]:
g1=diff(z,y,y)+diff(z,y)/(2*y)
g2=diff(w,x,x)+(4*y**2)*diff(w,y)-(8*y**2)*diff(z,x)-8*y*w
g3=diff(w,x,y)-diff(z,x,x)/2-diff(w,x)/(2*y)-diff(z,y)*6*y**2
g4=diff(w,y,y)-2*diff(z,x,y)-diff(w,y)/(2*y)-w/(2*y**2)

In [15]:
c2=Janet_Basis ([g1,g2,g3,g4], ctx)

******************   Autoreduce ****************************************
[<delierium.JanetBasis.DTerm object at 0x7efbd8ce0430>, <delierium.JanetBasis.DTerm object at 0x7efbd3b23a60>]


1/2*diff(z(x, y), y)/y + diff(z(x, y), y, y)

[<delierium.JanetBasis.DTerm object at 0x7efbe0fd4130>, <delierium.JanetBasis.DTerm object at 0x7efbd8ce0880>, <delierium.JanetBasis.DTerm object at 0x7efbd8ce0070>, <delierium.JanetBasis.DTerm object at 0x7efbd8c2de50>]


-1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x, y)

[<delierium.JanetBasis.DTerm object at 0x7efbda526370>, <delierium.JanetBasis.DTerm object at 0x7efbdf4eabe0>, <delierium.JanetBasis.DTerm object at 0x7efbda5260a0>, <delierium.JanetBasis.DTerm object at 0x7efbd9910250>]


...

******************   ReduceIntegr ****************************************


1/2*diff(z(x, y), y)/y + diff(z(x, y), y, y)

-1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x, y)

-6*y^2*diff(z(x, y), y) - 1/2*diff(w(x, y), x)/y + diff(w(x, y), x, y) - 1/2*diff(z(x, y), x, x)

4*y^2*diff(w(x, y), y) - 8*y^2*diff(z(x, y), x) - 8*y*w(x, y) + diff(w(x, y), x, x)

In [16]:
c2.show()

4*y^2*diff(w(x, y), y) - 8*y^2*diff(z(x, y), x) - 8*y*w(x, y) + diff(w(x, y), x, x)

4*y^2*diff(w(x, y), y) - 8*y^2*diff(z(x, y), x) - 8*y*w(x, y) + diff(w(x, y), x, x)

-6*y^2*diff(z(x, y), y) - 1/2*diff(w(x, y), x)/y + diff(w(x, y), x, y) - 1/2*diff(z(x, y), x, x)

-6*y^2*diff(z(x, y), y) - 1/2*diff(w(x, y), x)/y + diff(w(x, y), x, y) - 1/2*diff(z(x, y), x, x)

-1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x, y)

-1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x, y)

1/2*diff(z(x, y), y)/y + diff(z(x, y), y, y)

1/2*diff(z(x, y), y)/y + diff(z(x, y), y, y)